In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
%matplotlib Inline

In [3]:
df_full=pd.read_csv('../input/home-data-for-ml-course/train.csv', index_col='Id')
df_test_full=pd.read_csv('../input/home-data-for-ml-course/test.csv', index_col='Id')


In [4]:
df_full.shape

(1460, 80)

In [5]:
df_test_full.shape

(1459, 79)

In [6]:
df_full.isnull().sum()
    

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

In [7]:
df_test_full.isnull().sum()

MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
Street             0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 79, dtype: int64

In [8]:
df_full.dropna(axis=0,subset=['SalePrice'],inplace=True)
y=df_full.SalePrice
df_full.drop(['SalePrice'],axis=1,inplace=True)

In [9]:
X=df_full.select_dtypes(exclude=['object'])
X_test=df_test_full.select_dtypes(exclude=['object'])


In [10]:
X_tr,X_t,y_tr,y_t=train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
X_tr.isnull().sum()

MSSubClass         0
LotFrontage      212
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         6
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       58
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [12]:
def score_molde(X_tr,X_t,y_tr,y_t):
    model=RandomForestRegressor(n_estimators=100,random_state=0)
    model.fit(X_tr,y_tr)
    preds=model.predict(X_t)
    return mean_absolute_error(preds,y_t)

In [13]:
missing_col=[col for col in X_tr.columns
            if X_tr[col].isnull().any()]
X_tr_drop=X_tr.drop(missing_col,axis=1)
X_t_drop=X_t.drop(missing_col,axis=1)

In [14]:
score_molde(X_tr_drop,X_t_drop,y_tr,y_t)

17837.82570776256

In [15]:
from sklearn.impute import SimpleImputer
impute=SimpleImputer()
impute_X_tr=pd.DataFrame(impute.fit_transform(X_tr))
impute_X_t=pd.DataFrame(impute.transform(X_t))

In [16]:
score_molde(impute_X_tr,impute_X_t,y_tr,y_t)

18062.894611872147

In [17]:
from sklearn.impute import SimpleImputer
impute=SimpleImputer(strategy='median')
final_impute_X_tr=pd.DataFrame(impute.fit_transform(X_tr))
final_impute_X_t=pd.DataFrame(impute.transform(X_t))

In [18]:
score_molde(final_impute_X_tr,final_impute_X_t,y_tr,y_t)

17791.59899543379

In [19]:
final_X_t=pd.DataFrame(impute.transform(X_test))
model=RandomForestRegressor(n_estimators=100,random_state=0)
model.fit(final_impute_X_tr,y_tr)
pred=model.predict(final_X_t)
pred

array([125985.5 , 154599.5 , 180070.24, ..., 154751.44, 107387.  ,
       229281.55])